In [2]:
import pandas as pd
import plotly.graph_objects as go

In [3]:
def show_data(df,sensor,title="",labels=None,color="LightSalmon"):

  # Create figure
  fig = go.Figure()


  for c in df.columns:
    if sensor in c:
      fig.add_trace(
        go.Scatter(x=df.index.values, y=df[c].values,name=c)
        )

  # Set title
  fig.update_layout(
      title_text=title,
      width=900,
      height=400
  )

  # Add range slider
  fig.update_layout(
      xaxis=go.layout.XAxis(
          rangeslider=dict(
              visible=True
          ),
          type="linear"
      )
  )



  # plotting the labels
  if labels is not None:
    labels_shape = []
    for i in labels.index:
      labels_shape += [go.layout.Shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=str(labels.loc[i].Start),
            y0=0,
            x1=str(labels.loc[i].End),
            y1=1,
            fillcolor=color,
            opacity=0.5,
            layer="below",
            line_width=2,
        )]

    fig.update_layout(shapes=labels_shape)

  fig.show()                               

# Punch Data

In [4]:
punch_data =pd.read_csv("/content/data/punch.csv",skip_blank_lines=False)
punch_data["gesture_no"] = punch_data.isnull().all(axis=1).cumsum()
punch_data = punch_data.dropna()
punch_data.head()

,aX,aY,aZ,gX,gY,gZ,gesture_no
0,1.218,-0.524,0.928,-120.117,153.503,10.254,0
1,1.244,-0.595,0.894,-128.601,168.091,17.700,0
2,1.259,-0.656,0.889,-138.245,186.462,25.574,0
3,1.213,-0.763,0.814,-147.217,206.421,34.729,0
4,1.205,-0.837,0.678,-158.813,228.699,46.875,0


In [5]:
punch_data[punch_data.gesture_no == 6].max()

aX              1.229
aY              1.594
aZ              1.104
gX            290.649
gY            302.551
gZ            390.747
gesture_no      6.000
dtype: float64

In [6]:
show_data(punch_data[punch_data.gesture_no == 6],"a","Punch - Accel")

In [7]:
punch_features = pd.pivot_table(punch_data,index="gesture_no",aggfunc=["max"])
punch_features.columns = punch_features.columns.to_series().str.join('_')
punch_features["type"] = "punch"

# Flex Data

In [8]:
flex_data =pd.read_csv("/content/data/flex.csv",skip_blank_lines=False)
flex_data["gesture_no"] = flex_data.isnull().all(axis=1).cumsum()
flex_data = flex_data.dropna()
flex_data.head()

,aX,aY,aZ,gX,gY,gZ,gesture_no
0,0.741,0.606,-1.247,2.869,5.127,4.944,0
1,0.748,0.572,-1.236,3.784,-12.817,-2.441,0
2,0.746,0.528,-1.193,2.869,-31.433,-10.681,0
3,0.720,0.487,-1.143,1.282,-49.805,-18.372,0
4,0.715,0.466,-1.070,0.549,-66.833,-24.170,0


In [9]:
flex_data[flex_data.gesture_no == 6].max()

aX              0.720
aY              0.662
aZ              2.113
gX            147.827
gY            303.467
gZ             78.491
gesture_no      6.000
dtype: float64

In [10]:
show_data(flex_data[flex_data.gesture_no == 6],"a","Flex")

In [11]:
flex_features = pd.pivot_table(flex_data,index="gesture_no",aggfunc=["max"])
flex_features.columns = flex_features.columns.to_series().str.join('_')
flex_features["type"] = "flex"

In [12]:
features = pd.concat([flex_features,punch_features],ignore_index=True)
features

,max_aX,max_aY,max_aZ,max_gX,max_gY,max_gZ,type
0,0.759,0.606,1.729,25.146,61.707,27.283,flex
1,0.719,0.650,2.405,87.341,349.182,69.275,flex
2,0.570,0.464,2.250,84.412,304.687,88.928,flex
3,0.080,-0.068,1.391,33.752,24.536,5.005,flex
4,0.516,0.599,2.087,49.500,346.252,83.435,flex
...,...,...,...,...,...,...,...
57,0.994,2.338,1.310,237.427,392.517,334.290,punch
58,0.746,2.695,1.323,391.174,431.274,401.245,punch
59,0.731,1.525,1.375,260.437,420.471,330.688,punch
60,0.763,1.556,1.431,393.250,441.589,330.933,punch


In [13]:
import plotly.express as px
fig = px.scatter_matrix(features,
    dimensions=features.columns[:3],
    color="type")
fig.show()

# Machine Learning

In [22]:
#convert classifier to plain C code
!pip install micromlgen 
from micromlgen import port

In [23]:
from sklearn.ensemble import RandomForestClassifier
class_map = {0: "punch", 1: "flex"}
def get_classifier(features):
    features = features.replace(class_map[0], 0) #mapping punch class to 0
    features = features.replace(class_map[1], 1) #mapping flex class to 1
    X, y = features.values[:, :-1], features.values[:, -1]

    return RandomForestClassifier(20, max_depth=10).fit(X, y)

In [25]:

classifier = get_classifier(features)
c_code = port(classifier, classmap=class_map)

In [26]:
print(c_code)

#pragma once
#include <cstdarg>
namespace Eloquent {
    namespace ML {
        namespace Port {
            class RandomForest {
                public:
                    /**
                    * Predict class for features vector
                    */
                    int predict(float *x) {
                        uint8_t votes[2] = { 0 };
                        // tree #1
                        if (x[5] <= 173.79750442504883) {
                            votes[1] += 1;
                        }

                        else {
                            votes[0] += 1;
                        }

                        // tree #2
                        if (x[3] <= 227.84449768066406) {
                            votes[1] += 1;
                        }

                        else {
                            votes[0] += 1;
                        }

                        // tree #3
                        if (x[5] <= 178.58850479125977) {
                            